In [1]:
import warnings
import re
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pickle

In [2]:
!head -n 3 danbooru.csv

id,tags,created_at,uploader_id,source,score,up_score,down_score,favs,md5,file_size,file_ext,rating,is_status_locked,is_pending,is_flagged,is_deleted,width,height,has_children,parent_id,pools
263167,0:1girl 0:ankle_ribbon 5:artist_request 0:bangs 0:barefoot 0:blunt_bangs 0:brown_hair 0:eyes 0:hair_bun 0:hime_cut 0:japanese_clothes 0:long_hair 4:miyu_(vampire_princess_miyu) 0:ribbon 0:solo 3:vampire_princess_miyu 0:yellow_eyes,2008-06-10 03:16:23.64871 UTC,65792,,0,0,0,89268 68238,e5a4b492bc46d8ee9ac1dda94ed43387,52253,jpg,s,False,False,False,False,551,778,False,0,
332446,0:1boy 0:1girl 5:absurdres 0:animal_print 5:bad_id 5:bad_pixiv_id 0:bat_print 0:bat_wings 0:blood 0:demon_girl 1:harmaline 0:head_wings 5:highres 0:long_hair 4:morrigan_aensland 0:pantyhose 0:print_legwear 0:red_eyes 0:succubus 3:vampire_(game) 0:wings,2008-10-17 02:34:45.35441 UTC,13056,http://img04.pixiv.net/img/takana/1809424.jpg,0,0,0,,69fb3d60085c54b1e8573574c7433991,3912531,jpg,s,False,False,False,False,1600,3300,

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
import urllib.parse
face_tags = urllib.parse.unquote("angry anger_vein annoyed clenched_teeth blush blush_stickers embarrassed bored closed_eyes confused crazy disdain disgust drunk envy expressionless evil facepalm flustered frustrated grimace guilt happy kubrick_stare lonely nervous nosebleed one_eye_closed open_mouth closed_mouth parted_lips pain pout raised_eyebrow rape_face rolling_eyes sad depressed frown gloom_%28expression%29 tears horns scared panicking worried serious sigh sleepy tired sulking thinking pensive wince afterglow ahegao fucked_silly naughty_face torogao smile crazy_smile evil_smile fingersmile forced_smile glasgow_smile grin fang evil_grin light_smile sad_smile seductive_smile stifled_laugh smug doyagao smirk smug troll_face surprised scared %2F%5C%2F%5C%2F%5C color_drain horror_%28expression%29 screaming turn_pale trembling wavy_mouth %3B%29 %3Ad %3Bd xd d%3A %3A%7D %3A%7B %3A3 %3B3 x3 3%3A uwu %3D.w.%3D %3Ap %3Bp %3Aq %3Bq %3E%3A%29 %3E%3A%28 %3At %3Ai %3A%2F %3Ax %3Ac c%3A %3A%3C %3B%3C %3A%3C%3E %3A%3E %3A%3E%3D %3B%3E%3D %3Ao %3Bo %3D %3D%29 %3Dd %3Do %3Dv %7C3 %7Cd %7Co o3o %28-3-%29 %3E3%3C o_o 0_0 ._.  %E2%80%A2_%E2%80%A2 solid_circle_eyes %E2%99%A5_%E2%99%A5 heart_eyes %5E_%5E %5Eo%5E %5C%28%5Eo%5E%29%2F %E2%94%94%28%5Eo%5E%29%E2%94%90%E2%89%A1 %5Eq%5E %3E_%3C xd x3 %3Eo%3C %3C_%3E %3B_%3B %40_%40 %3E_%40 %2B_%2B %2B_- -_- %5C_%2F %3D_%3D %3D%5E%3D %3Dv%3D %3Co%3E_%3Co%3E constricted_pupils cross_eyed rectangular_mouth sideways_mouth no_nose no_mouth wavy_mouth wide-eyed mouth_drool awesome_face foodgasm henohenomoheji nonowa portrait profile smiley_face uso_da food_awe breast_awe penis_awe").split()
eye_tags = urllib.parse.unquote("aqua_eyes black_eyes blue_eyes brown_eyes green_eyes grey_eyes orange_eyes lavender_eyes pink_eyes purple_eyes red_eyes silver_eyes white_eyes yellow_eyes heterochromia multicolored_eyes al_bhed_eyes pac-man_eyes ringed_eyes constricted_pupils dilated_pupils horizontal_pupils no_pupils slit_pupils symbol-shaped_pupils %2B_%2B heart-shaped_pupils star-shaped_pupils blue_sclera black_sclera blank_eyes bloodshot_eyes green_sclera mismatched_sclera orange_sclera red_sclera yellow_sclera bags_under_eyes bruised_eye flaming_eyes glowing_eyes glowing_eye mako_eyes amphibian_eyes button_eyes cephalopod_eyes compound_eyes frog_eyes crazy_eyes empty_eyes heart_eyes nonowa solid_circle_eyes o_o 0_0 jitome tareme tsurime sanpaku sharingan mangekyou_sharingan eye_reflection text_in_eyes missing_eye one-eyed third_eye extra_eyes no_eyes").split()
eye_expressions = urllib.parse.unquote("%3E_%3C x3 xd o_o 0_0 3_3 6_9 %40_%40 %5E_%5E %5Eo%5E 9848 26237 %3D_%3D %2B_%2B ._.  %3Co%3E_%3Co%3E blinking closed_eyes wince one_eye_closed %3B%3C %3B%3E %3Bp").split()
eye_other = urllib.parse.unquote("covering_eyes hair_over_eyes hair_over_one_eye bandage_over_one_eye blindfold hat_over_eyes eyepatch eyelashes colored_eyelashes fake_eyelashes eyes_visible_through_hair glasses makeup eyeliner eyeshadow mascara eye_contact looking_afar looking_at_another looking_at_breasts looking_at_hand looking_at_mirror looking_at_phone looking_at_viewer looking_away looking_back looking_down looking_out_window looking_over_glasses looking_through_legs looking_to_the_side looking_up akanbe blind cross-eyed drawn_on_eyes eyeball eye_beam eye_poke eye_pop persona_eyes shading_eyes squinting staring uneven_eyes upturned_eyes wall-eyed wide-eyed wince").split()
ears_tags = urllib.parse.unquote("animal_ears bear_ears bunny_ears cat_ears dog_ears fake_animal_ears fox_ears horse_ears kemonomimi_mode lion_ears monkey_ears mouse_ears raccoon_ears sheep_ears tiger_ears wolf_ears pointy_ears robot_ears extra_ears ear_piercing ear_protection earrings single_earring headphones covering_ears ear_biting ear_licking ear_grab").split()
hair_tags = urllib.parse.unquote("heartbreak_haircut hand_in_hair adjusting_hair bunching_hair hair_flip hair_grab hair_pull hair_tucking hair_tousle hair_twirling hair_sex hair_brush hair_dryer shampoo bun_cover hairpods chopsticks comb hair_ornament hair_bell hair_bobbles hair_bow hair_ribbon hairclip hairpin hair_flower hair_tubes kanzashi hair_tie hairband hair_weapon headband scrunchie wig facial_hair beard bearded_girl goatee mustache fake_mustache stubble fiery_hair prehensile_hair helicopter_hair tentacle_hair living_hair detached_hair severed_hair floating_hair hair_spread_out wet_hair").split()
hair_color_tags = urllib.parse.unquote("aqua_hair black_hair blonde_hair blue_hair light_blue_hair brown_hair light_brown_hair green_hair grey_hair magenta_hair orange_hair pink_hair purple_hair lavender_hair red_hair auburn_hair maroon_hair silver_hair white_hair multicolored_hair colored_inner_hair gradient_hair rainbow_hair streaked_hair two-tone_hair highlights colored_tips alternate_hair_color").split()
hair_style_tags = urllib.parse.unquote("very_short_hair short_hair medium_hair long_hair very_long_hair absurdly_long_hair big_hair bald bald_girl alternate_hairstyle hair_down hair_up curly_hair drill_hair twin_drills flipped_hair hair_flaps messy_hair pointy_hair ringlets spiked_hair wavy_hair bangs asymmetrical_bangs blunt_bangs hair_over_eyes hair_over_one_eye parted_bangs swept_bangs hair_between_eyes hair_intakes sidelocks widow%27s_peak ahoge heart_ahoge huge_ahoge antenna_hair comb_over hair_pulled_back hair_slicked_back mohawk hair_bikini hair_censor hair_in_mouth hair_over_breasts hair_over_one_breast hair_over_crotch hair_over_shoulder hair_scarf bow_by_hair braid braided_bangs front_braid side_braid french_braid crown_braid single_braid multiple_braids twin_braids tri_braids quad_braids hair_bun braided_bun double_bun triple_bun hair_rings half_updo one_side_up two_side_up low-braided_long_hair low-tied_long_hair mizura multi-tied_hair nihongami ponytail folded_ponytail front_ponytail high_ponytail short_ponytail side_ponytail split_ponytail topknot twintails low_twintails short_twintails uneven_twintails tri_tails quad_tails quin_tails bob_cut bowl_cut buzz_cut chonmage crew_cut flattop pixie_cut undercut cornrows hairlocs hime_cut mullet afro huge_afro beehive_hairdo pompadour quiff shouten_pegasus_mix_mori").split()
skin_color_tags = urllib.parse.unquote("dark_skin pale_skin tan tanlines sun_tattoo black_skin blue_skin green_skin grey_skin orange_skin pink_skin purple_skin red_skin white_skin yellow_skin shiny_skin").split()
headwear_tags = urllib.parse.unquote("crown hat helmet black_headwear blue_headwear brown_headwear green_headwear grey_headwear orange_headwear pink_headwear purple_headwear red_headwear white_headwear yellow_headwear ajirogasa animal_hat cat_hat penguin_hat baseball_cap beanie beret bicorne boater_hat bowl_hat bowler_hat bucket_hat cabbie_hat chef_hat toque_blanche flat_top_chef_hat cloche_hat cowboy_hat deerstalker deviruchi_hat dixie_cup_hat eggshell_hat fedora female_service_cap flat_cap fur_hat garrison_cap jester_cap kepi mian_guan mitre mob_cap mortarboard nightcap nurse_cap party_hat peaked_cap pillow_hat pirate_hat porkpie_hat pumpkin_hat rice_hat robe_and_wizard_hat sailor_hat santa_hat mini_santa_hat shako_cap shampoo_hat sombrero sun_hat tam_o%27_shanter tate_eboshi tokin_hat top_hat mini_top_hat tricorne ushanka witch_hat mini_witch_hat wizard_hat veil zun_hat baseball_helmet bicycle_helmet brodie_helmet diving_helmet football_helmet hardhat horned_helmet helm kabuto motorcycle_helmet pickelhaube pith_helmet stahlhelm tank_helmet winged_helmet circlet diadem mini_crown saishi tiara aviator_cap bandana bonnet dalachi_%28headdress%29 habit hijab keffiyeh shower_cap visor_cap checkered_hat frilled_hat military_hat mini_hat multicolored_hat police_hat print_hat school_hat straw_hat adjusting_hat hand_on_headwear hands_on_headwear hat_basket hat_loss hat_on_chest hat_over_eyes hat_over_one_eye hat_removed hat_tip holding_hat torn_hat no_hat hat_bow hat_feather hat_flower hat_ribbon hat_with_ears adjusting_hat backwards_hat hat_removed holding_hat torn_hat hair_bow hair_ribbon hairband headband forehead_protector sweatband hachimaki nejiri_hachimaki mongkhon headdress maid_headdress veil hood").split()
neckwear_tags = urllib.parse.unquote("collarbone nape neck long_neck arm_around_neck arms_around_neck hand_on_another%27s_neck hand_on_own_neck neck_biting necking neck_kiss ascot bolo_tie bowtie choker bell_choker ribbon_choker collar bell_collar dog_collar spiked_collar studded_collar cravat elizabethan_collar feather_boa goggles_around_neck headphones_around_neck lanyard neckerchief necklace chain_necklace flower_necklace lei friendship_charm pearl_necklace pendant amulet locket magatama pentacle neck_ribbon neck_ruff necktie necktie_between_breasts necktie_on_head necktie_grab necktie_removed scarf stole x-shaped_choker ascot bowtie choker neck_ribbon neckerchief necktie aqua_neckwear black_neckwear blue_neckwear brown_neckwear green_neckwear grey_neckwear orange_neckwear pink_neckwear purple_neckwear red_neckwear white_neckwear yellow_neckwear checkered_neckwear plaid_neckwear striped_neckwear collar_grab collar_tug adjusting_collar collared_shirt detached_collar fur_collar high_collar open_collar popped_collar sailor_collar turtleneck sleeveless_turtleneck v-neck wing_collar").split()
eyewear_tags = urllib.parse.unquote("glasses monocle sunglasses aqua-framed_eyewear black-framed_eyewear blue-framed_eyewear brown-framed_eyewear green-framed_eyewear grey-framed_eyewear orange-framed_eyewear pink-framed_eyewear purple-framed_eyewear red-framed_eyewear white-framed_eyewear yellow-framed_eyewear blue-tinted_eyewear brown-tinted_eyewear green-tinted_eyewear orange-tinted_eyewear pink-tinted_eyewear purple-tinted_eyewear red-tinted_eyewear yellow-tinted_eyewear heart-shaped_eyewear round_eyewear over-rim_eyewear rimless_eyewear semi-rimless_eyewear under-rim_eyewear adjusting_eyewear eyewear_on_head eyewear_removed eyewear_hang eyewear_in_mouth holding_eyewear eyewear_strap eyewear_switch looking_over_eyewear no_eyewear 3d_glasses coke-bottle_glasses diving_mask fancy_glasses heart-shaped_eyewear funny_glasses goggles nodoka_glasses opaque_glasses pince-nez safety_glasses shooting_glasses ski_goggles x-ray_glasses bespectacled kamina_shades star_shades").split()
piercings_tags = urllib.parse.unquote("ear_piercing eyebrow_piercing anti-eyebrow_piercing eyelid_piercing lip_piercing labret_piercing nose_piercing bridge_piercing tongue_piercing").split()
format_tags = urllib.parse.unquote("3d animated animated_png flash music_video song video animated_gif non-looping_animation archived_file artbook bmp calendar_%28medium%29 card_%28medium%29 comic 2koma 3koma 4koma multiple_4koma 5koma borderless_panels doujinshi eromanga left-to-right_manga right-to-left_comic silent_comic corrupted_image cover album_cover character_single cover_page doujin_cover dvd_cover fake_cover game_cover magazine_cover manga_cover fake_screenshot game_cg gyotaku_%28medium%29 highres absurdres incredibly_absurdres lowres thumbnail huge_filesize icon logo kirigami lineart no_lineart outline long_image tall_image wide_image mosaic_art photomosaic oekaki official_art phonecard photo papercraft paper_child paper_cutout pixel_art postcard poster revision bad_revision artifacted_revision censored_revision corrupted_revision lossy_revision watermarked_revision scan screencap shitajiki tegaki transparent_background triptych_%28art%29 vector_trace wallpaper dual_monitor ios_wallpaper official_wallpaper phone_wallpaper psp_wallpaper tileable wallpaper_forced widescreen").split()
style_tags = urllib.parse.unquote("abstract art_deco art_nouveau fine_art_parody flame_painter impressionism nihonga sumi-e ukiyo-e minimalism realistic photorealistic sketch style_parody list_of_style_parodies surreal traditional_media faux_traditional_media work_in_progress backlighting blending bloom bokeh caustics chiaroscuro chromatic_aberration chromatic_aberration_abuse diffraction_spikes depth_of_field dithering drop_shadow emphasis_lines foreshortening gradient halftone lens_flare lens_flare_abuse motion_blur motion_lines multiple_monochrome optical_illusion anaglyph exif_thumbnail_surprise open_in_internet_explorer open_in_winamp stereogram scanlines silhouette speed_lines vignetting").split()
all_tags = face_tags+eye_tags+eye_expressions+eye_other+ears_tags+hair_tags+hair_color_tags+hair_style_tags+skin_color_tags+headwear_tags+eyewear_tags+piercings_tags+format_tags+style_tags


In [ ]:
#count = 0
import csv
class MyCorpus(object):
    def __init__(self):
        self.count = 0
    def __iter__(self):
        with open('danbooru.csv', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                self.count = self.count + 1
                #fields = line.split(",")
                index = int(row['id'])
                tags = str(row['tags']).split()
                # assume there's one document per line, tokens separated by whitespace
                if self.count % 5000 == 0:
                    print(self.count, index)
                words = []
                for tag in tags:
                    if tag.split(":")[0] != '0' or (tag.split(":")[1] in all_tags):
                        words.append(tag)
                yield TaggedDocument(words=words, tags=[index])

In [ ]:
# instantiate Doc2Vec model

max_epochs = 20
vec_size = 128
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=100,
                max_vocab_size=1000000,
                #docvecs_mapfile='danbooru/mapfile1.txt',
                #verbose=True,
                workers=7,
                dm=0) # paragraph vector distributed bag-of-words (PV-DBOW)
  


In [ ]:
corpus = MyCorpus()
print(corpus)
model.build_vocab(corpus, progress_per=10000)

In [ ]:
# train Doc2Vec model
# stochastic (random initialization), so each run will be different unless you specify seed
from gensim.test.utils import get_tmpfile
fname = "danbooru/danbooru_subset_tags_128.d2v"
print('Epoch', end = ': ')
for epoch in range(max_epochs):
    print(epoch, end = ' ')
    model.train(corpus,
              total_examples=model.corpus_count,
              epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    model.save(fname)


In [ ]:
len(all_tags)

# Load model, test with some vectors

In [ ]:
import pandas as pd
import numpy as np
import umap
import umap.plot
import matplotlib.pyplot as plt

In [ ]:
fname = "danbooru/danbooru_subset_tags_128.d2v"
model = Doc2Vec.load(fname)

In [ ]:
vecs = model.docvecs.vectors_docs

In [ ]:
model.docvecs[263167]

In [ ]:
test_tags = "0:1girl 0:ankle_ribbon 5:artist_request 0:bangs 0:barefoot 0:blunt_bangs 0:brown_hair 0:eyes 0:hair_bun 0:hime_cut 0:japanese_clothes 0:long_hair 4:miyu_(vampire_princess_miyu) 0:ribbon 0:solo 3:vampire_princess_miyu 0:yellow_eyes".split()

In [ ]:
test_words = []
for tag in test_tags:
    if tag.split(":")[0] != '0' or (tag.split(":")[1] in all_tags):
        test_words.append(tag)
test_words

In [ ]:
vector = model.infer_vector(test_words)
vector

In [ ]:
sims = model.docvecs.most_similar([vector])

In [ ]:
sims

In [ ]:
holo_vec = model.infer_vector(["0:animal_ears", "0:red_eyes", "0:wolf_ears"])
holo_vec

In [ ]:
holo_sims = model.docvecs.most_similar([holo_vec])
holo_sims

# Get just the vectors we care about

In [ ]:
import glob
import sys
import os
def _get_all_files(path):
    if os.path.isfile(path):
        return [path]

    possible_files = sorted(glob.glob(os.path.join(path, "*")))
    return_list = []
    for possible_file in possible_files:
        return_list.extend(_get_all_files(possible_file))
    return return_list

In [ ]:
portrait_files = _get_all_files("portraits")
portrait_files

In [ ]:
A = model.docvecs[1000031]
for file in portrait_files[1:]:
    f = file.split("/")[-1].split(".")[0]
    index = int(f[:len(f)-1])
    print(file, index)
    vec = model.docvecs[index]
    A = np.vstack([A, vec])

In [ ]:
A.shape

In [ ]:
np.save("portraits/labels.npy", A)